In [36]:
!pip install qgrid
import qgrid
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler # Escalamiento estándar
from sklearn.model_selection import train_test_split   
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import confusion_matrix 
import scipy as sc
import math
from numpy import random
from numpy import matlib
from scipy.spatial import distance
import matplotlib.pyplot as plt






In [37]:
#Eliminamos las filas nulas
DatosConNulls = pd.read_csv("water_potability.csv", names = None, sep = ",")


In [38]:
DatosConNulls.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

In [39]:
#completar los valores en null con su media
df1=DatosConNulls[DatosConNulls['Potability']==1].copy()
df2=DatosConNulls[DatosConNulls['Potability']==0].copy()
df1['ph']=df1['ph'].replace(np.nan, df1['ph'].median())
df2['ph']=df2['ph'].replace(np.nan, df2['ph'].median())
df1['Sulfate']=df1['Sulfate'].replace(np.nan, df1['Sulfate'].median())
df2['Sulfate']=df2['Sulfate'].replace(np.nan, df2['Sulfate'].median())
df1['Trihalomethanes']=df1['Trihalomethanes'].replace(np.nan, df1['Trihalomethanes'].median())
df2['Trihalomethanes']=df2['Trihalomethanes'].replace(np.nan, df2['Trihalomethanes'].median())
Datos=pd.concat([df1,df2], axis=0, ignore_index=True)
Datos.isnull().sum()

ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
Potability         0
dtype: int64

In [40]:
X = Datos.iloc[:,0:9] #Caracteristicas
Nombres = Datos.columns
Y = Datos['Potability']#clases
#Ya que la diferencia entre las escalas de los datos de X, se hará una normalización de los datos 
scaler = StandardScaler()
scaler.fit(X)
X_escalado = scaler.transform(X)


In [41]:
def completarTablaRedesNeuronales (i,tablaRedesNeuronales,EV,IC,F1,ICF1):
  tablaRedesNeuronales.loc[i,"Eficiencia en validacion"] = str(EV) # reemplazar los valores
  tablaRedesNeuronales.loc[i,"Intervalo de confianza de eficiencia"] = str(IC)
  tablaRedesNeuronales.loc[i,"f1-score"] = str(F1)
  tablaRedesNeuronales.loc[i,"Intervalo de confianza de f1-score"] = str(ICF1)
  return(tablaRedesNeuronales)

In [53]:
def completarTablaSoporteVectorial (i,tablaSoporteVectorial,EV,IC,PV,F1,ICF1):
  tablaSoporteVectorial.loc[i,"Eficiencia en validacion"] = str(EV)
  tablaSoporteVectorial.loc[i,"Intervalo de confianza de eficiencia"] = str(IC)
  tablaSoporteVectorial.loc[i, "% de Vectores de Soporte"] = str(PV)
  tablaSoporteVectorial.loc[i,"f1-score"] = str(F1)
  tablaSoporteVectorial.loc[i,"Intervalo de confianza de f1-score"] = str(ICF1)
  return(tablaSoporteVectorial)

In [62]:
def completarTablaBoostingTree (pos,tablaBoostingTree,EV,IC,n_estimators,learning_rate,max_depth,F1,ICF1):
  tablaBoostingTree.loc[pos,"n_estimators"]=n_estimators
  tablaBoostingTree.loc[pos,"learning_rate"]=learning_rate
  tablaBoostingTree.loc[pos,"max_depth"]=max_depth
  tablaBoostingTree.loc[pos,"Eficiencia en validacion"] = str(EV) 
  tablaBoostingTree.loc[pos,"Intervalo de confianza"] = str(IC)
  tablaBoostingTree.loc[pos,"f1-score"] = str(F1)
  tablaBoostingTree.loc[pos,"Intervalo de confianza de f1-score"] = str(ICF1)
  return(tablaBoostingTree)

In [51]:
def completarTablaAnalisisDiscriminanteCuadratico (i,tablaAnalisisDiscriminanteCuadratico,EV,IC,F1,ICF1):
  tablaAnalisisDiscriminanteCuadratico.loc[i,"Eficiencia en validacion"] = str(EV) 
  tablaAnalisisDiscriminanteCuadratico.loc[i,"Intervalo de confianza de eficiencia"] = str(IC)
  tablaAnalisisDiscriminanteCuadratico.loc[i,"f1-score"] = str(F1)
  tablaAnalisisDiscriminanteCuadratico.loc[i,"Intervalo de confianza de f1-score"] = str(ICF1)
  return(tablaAnalisisDiscriminanteCuadratico)

In [50]:
def completarTablaVentanaParzen (i,tablaVentanaParzen,EV,IC,F1,ICF1):
  tablaVentanaParzen.loc[i,"Eficiencia en validacion"] = str(EV) 
  tablaVentanaParzen.loc[i,"Intervalo de confianza de eficiencia"] = str(IC)
  tablaVentanaParzen.loc[i,"f1-score"] = str(F1)
  tablaVentanaParzen.loc[i,"Intervalo de confianza de f1-score"] = str(ICF1)
  return(tablaVentanaParzen)

# 1. Redes neuronales

In [ ]:
 #Integer encode
label_encoder = LabelEncoder()
integerY = label_encoder.fit_transform(Y)
print(integerY)

#Binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integerY = integerY.reshape(len(integerY), 1)
onehot_encoded = onehot_encoder.fit_transform(integerY)
print(onehot_encoded)

[1 1 1 ... 0 0 0]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
import scipy as sc
import math
from numpy import random
from numpy import matlib


tablaRedesNeuronales = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2,5,5,5,5,5,6,6,6,6,6]),
    'Neuronas por capa' : pd.Series([2,4,10,28,32,2,4,10,28,32,2,4,10,28,32,2,4,10,28,32])})

numerocapas = [1,1,1,1,1,2,2,2,2,2,5,5,5,5,5,6,6,6,6,6]
neuronascapa = [2,4,10,28,32,2,4,10,28,32,2,4,10,28,32,2,4,10,28,32]

for i in range (20):

  Folds = 4
  random.seed(19680801)
  EficienciaTrain = np.zeros(Folds)
  EficienciaVal = np.zeros(Folds)
  f1 = np.zeros(Folds)
  skf = StratifiedKFold(n_splits=Folds)
  j = 0
  for train, test in skf.split(X_escalado, integerY):
      Xtrain = X_escalado[train,:]
      Ytrain = integerY[train]
      Xtest = X_escalado[test,:]
      Ytest = integerY[test]
      Ytest=Ytest.reshape(len(Ytest))
      Ytrain=Ytrain.reshape(len(Ytrain))
      
    
      if (numerocapas[i]==1):
        vector=(neuronascapa[i])
      if (numerocapas[i]==2):
        vector=(neuronascapa[i],neuronascapa[i])
      if (numerocapas[i]==5):
        vector=(neuronascapa[i],neuronascapa[i],neuronascapa[i],neuronascapa[i],neuronascapa[i])
      if (numerocapas[i]==6):
        vector=(neuronascapa[i],neuronascapa[i],neuronascapa[i],neuronascapa[i],neuronascapa[i],neuronascapa[i])  

      #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
      mlp = MLPClassifier(hidden_layer_sizes=vector, activation='identity', max_iter=400,verbose=True)
      mlp.out_activation_='softmax'
      mlp=mlp.fit(Xtrain,Ytrain)

      Ytrain_pred = mlp.predict(Xtrain)
      Yest = mlp.predict(Xtest)

      
      #Evaluamos las predicciones del modelo con los datos de test
      EficienciaTrain[j] = metrics.accuracy_score(Ytrain, Ytrain_pred)
      EficienciaVal[j] = metrics.accuracy_score(Ytest, Yest)
      f1[j]=metrics.f1_score(Ytest, Yest,average='weighted')
      j += 1
          
  print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
  print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
  print("f1-score ",str(np.mean(f1))," +- ",np.std(f1))
  tablaRedesNeuronales = completarTablaRedesNeuronales(i,tablaRedesNeuronales,np.mean(EficienciaVal),np.std(EficienciaVal),np.mean(f1),np.std(f1))

Iteration 1, loss = 0.73947586
Iteration 2, loss = 0.73255183
Iteration 3, loss = 0.72630518
Iteration 4, loss = 0.72084060
Iteration 5, loss = 0.71577958
Iteration 6, loss = 0.71148337
Iteration 7, loss = 0.70738935
Iteration 8, loss = 0.70374855
Iteration 9, loss = 0.70049213
Iteration 10, loss = 0.69748388
Iteration 11, loss = 0.69489640
Iteration 12, loss = 0.69234934
Iteration 13, loss = 0.69010342
Iteration 14, loss = 0.68813002
Iteration 15, loss = 0.68630076
Iteration 16, loss = 0.68453447
Iteration 17, loss = 0.68294305
Iteration 18, loss = 0.68148567
Iteration 19, loss = 0.68019911
Iteration 20, loss = 0.67909370
Iteration 21, loss = 0.67792273
Iteration 22, loss = 0.67677033
Iteration 23, loss = 0.67583675
Iteration 24, loss = 0.67500774
Iteration 25, loss = 0.67415089
Iteration 26, loss = 0.67344947
Iteration 27, loss = 0.67275346
Iteration 28, loss = 0.67219157
Iteration 29, loss = 0.67172219
Iteration 30, loss = 0.67123879
Iteration 31, loss = 0.67081891
Iteration 32, los

In [ ]:
tablaRedesNeuronales.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)
qgrid_widget = qgrid.show_grid(tablaRedesNeuronales, show_toolbar=False)
qgrid_widget
qgrid_widget.get_changed_df()

Eficiencia en validacion  ... Intervalo de confianza de f1-score
N. de capas ocultas Neuronas por capa                           ...                                   
1                   2                       0.6111111111111112  ...               0.004030014347053157
                    4                       0.6117216117216118  ...              0.0055689576547940794
                    10                      0.6117216117216118  ...              0.0055689576547940794
                    28                      0.6111111111111112  ...                0.00660342592653192
                    32                      0.6114163614163615  ...                0.00686333242118209
2                   2                       0.6098901098901099  ...              0.0014649035750146594
                    4                       0.6108058608058609  ...              0.0040787518209848455
                    10                      0.6105006105006106  ...                0.00514995462331914
                    28                      0.6120268620268621  ...               0.006723795777252328
                    32                      0.6117216117216118  ...               0.009376664185810379
5                   2                       0.6101953601953602  ...              0.0026547837987329764
                    4                       0.6105006105006106  ...               0.002904242941163179
                    10                      0.6105006105006104  ...               0.004132832467887253
                    28                      0.6117216117216118  ...               0.006274752834710863
                    32                      0.6105006105006106  ...               0.004236176401888313
6                   2                       0.6098901098901099  ...              0.0007498897315181274
                    4                       0.6098901098901099  ...              0.0007498897315181274
                    10                      0.6111111111111112  ...               0.007065254843499386
                    28                      0.6098901098901099  ...               0.005107749023790897
                    32                      0.6105006105006106  ...              0.0032595080248605758

[20 rows x 4 columns]

# 2. Máquinas de soporte vectorial

In [ ]:
import sklearn
from sklearn import preprocessing

#Se modifica cada etiqueta para que corresponda a un numero

dataSet=[]
label = np.unique(Y)
encoder=preprocessing.LabelEncoder()
encoder.fit(label)
data=encoder.transform(Y)
dataSet.append(data)
dataFrameY=pd.DataFrame(dataSet).values
dataFrameY=dataFrameY.T
dataFrameY = dataFrameY.astype(np.float)
dataFrameY=dataFrameY.ravel() 

In [ ]:
import math
import numpy as np
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import qgrid

tablaSoporteVectorial = pd.DataFrame({
    'Kernel' : pd.Series(['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})

k = ['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']
c = [0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]
g =  [0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1]


for i in range (24):

  #Validamos el modelo
  Folds = 4
  random.seed(24524)
  EficienciaTrain = np.zeros(Folds)
  EficienciaVal = np.zeros(Folds)
  skf = StratifiedKFold(n_splits=Folds)
  f1 = np.zeros(Folds)
  j = 0
  porcentaje = np.zeros(4)

  for train, test in skf.split(X_escalado, dataFrameY):
      Xtrain = X_escalado[train,:]
      Ytrain = dataFrameY[train]
      Xtest = X_escalado[test,:]
      Ytest = dataFrameY[test]
      
      #Normalizamos los datos
      scaler = preprocessing.StandardScaler().fit(Xtrain)
      Xtrain = scaler.transform(Xtrain)
      Xtest = scaler.transform(Xtest)
      
      if( k[i] == 'linear'):
        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        modelo = SVC(kernel= k[i], C=c[i], decision_function_shape='ovr')#Si es Lineal
      else:
        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        modelo = SVC(kernel= k[i], C=c[i], gamma=g[i],decision_function_shape='ovr') #Si es rbf
      
      modelo = modelo.fit(Xtrain,Ytrain)
      #Validación
      Ytrain_pred = modelo.predict(Xtrain)
      Yest = modelo.predict(Xtest)

      
      #Evaluamos las predicciones del modelo con los datos de test
      EficienciaTrain[j] = metrics.accuracy_score(Ytrain, Ytrain_pred)
      EficienciaVal[j] = metrics.accuracy_score(Ytest, Yest)
      f1[j]=metrics.f1_score(Ytest, Yest,average='weighted')
      porcentaje[j] =  len(modelo.support_vectors_) / len(Xtrain)
      
      j += 1
          
  print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
  print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
  print("f1-score ",str(np.mean(f1))," +- ",np.std(f1))
  tablaSoporteVectorial=completarTablaSoporteVectorial(i,tablaSoporteVectorial,np.mean(EficienciaVal),np.std(EficienciaVal),np.mean(porcentaje),np.mean(f1),np.std(f1))

Eficiencia durante el entrenamiento = 0.6098901098901098+-0.00020350020350018427
Eficiencia durante la validación = 0.6098901098901099+-0.0006105006105006638
f1-score  0.4621012175471858  +-  0.0007498897315181274
Eficiencia durante el entrenamiento = 0.6098901098901098+-0.00020350020350018427
Eficiencia durante la validación = 0.6098901098901099+-0.0006105006105006638
f1-score  0.4621012175471858  +-  0.0007498897315181274
Eficiencia durante el entrenamiento = 0.6098901098901098+-0.00020350020350018427
Eficiencia durante la validación = 0.6098901098901099+-0.0006105006105006638
f1-score  0.4621012175471858  +-  0.0007498897315181274
Eficiencia durante el entrenamiento = 0.6098901098901098+-0.00020350020350018427
Eficiencia durante la validación = 0.6098901098901099+-0.0006105006105006638
f1-score  0.4621012175471858  +-  0.0007498897315181274
Eficiencia durante el entrenamiento = 0.6098901098901098+-0.00020350020350018427
Eficiencia durante la validación = 0.6098901098901099+-0.000610

In [ ]:
tablaSoporteVectorial

,Kernel,C,gamma,Eficiencia en validacion,Intervalo de confianza de eficiencia,% de Vectores de Soporte,f1-score,Intervalo de confianza de f1-score
0,lineal,0.001,0.00,0.6098901098901099,0.0006105006105006638,0.7818477818477818,0.4621012175471858,0.0007498897315181274
1,lineal,0.010,0.00,0.6098901098901099,0.0006105006105006638,0.7831705331705332,0.4621012175471858,0.0007498897315181274
2,lineal,0.100,0.00,0.6098901098901099,0.0006105006105006638,0.78998778998779,0.4621012175471858,0.0007498897315181274
3,lineal,1.000,0.00,0.6098901098901099,0.0006105006105006638,0.8421855921855922,0.4621012175471858,0.0007498897315181274
4,lineal,10.000,0.00,0.6098901098901099,0.0006105006105006638,0.8810541310541311,0.4621012175471858,0.0007498897315181274
5,lineal,100.000,0.00,0.6098901098901099,0.0006105006105006638,0.8569393569393569,0.4621012175471858,0.0007498897315181274
6,rbf,0.001,0.01,0.6098901098901099,0.0006105006105006638,0.7802197802197803,0.4621012175471858,0.0007498897315181274
7,rbf,0.001,0.10,0.6098901098901099,0.0006105006105006638,0.7842897842897842,0.4621012175471858,0.0007498897315181274
8,rbf,0.001,1.00,0.6098901098901099,0.0006105006105006638,0.7894790394790395,0.4621012175471858,0.0007498897315181274
9,rbf,0.010,0.01,0.6098901098901099,0.0006105006105006638,0.7888685388685389,0.4621012175471858,0.0007498897315181274


# 3. Ventana de parzen

In [46]:
from sklearn.base import BaseEstimator, ClassifierMixin
class KDEClassifier(BaseEstimator, ClassifierMixin):
    """Bayesian generative classification based on KDE
    
    Parameters
    ----------
    bandwidth : float
        the kernel bandwidth within each class
    kernel : str
        the kernel name, passed to KernelDensity
    """
    def __init__(self, bandwidth=1.0, kernel='gaussian'):
        self.bandwidth = bandwidth
        self.kernel = kernel
        
    def fit(self, X, y):
        self.classes_ = np.sort(np.unique(y))
        training_sets = [X[y == yi] for yi in self.classes_]
        self.models_ = [KernelDensity(bandwidth=self.bandwidth,
                                      kernel=self.kernel).fit(Xi)
                        for Xi in training_sets]
        self.logpriors_ = [np.log(Xi.shape[0] / X.shape[0])
                           for Xi in training_sets]
        return self
        
    def predict_proba(self, X):
        logprobs = np.array([model.score_samples(X)
                             for model in self.models_]).T
        result = np.exp(logprobs + self.logpriors_)
        return result / result.sum(1, keepdims=True)
        
    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), 1)]

In [54]:
tablaVentanaParzen = pd.DataFrame({
    'bandwidth' : pd.Series([0.2,0.5,0.7,1, 2, 5, 10])})


bandwidth = [0.2,0.5,0.7,1, 2, 5, 10]
for i in range(7):
  clf = KDEClassifier(bandwidth[i])
  Folds = 4
  EficienciaTrain = np.zeros(Folds)
  EficienciaVal = np.zeros(Folds)
  f1 = np.zeros(Folds)
  skf = StratifiedKFold(n_splits=Folds)
  j = 0
  for train, test in skf.split(X_escalado, integerY):
      Xtrain = X_escalado[train,:]
      Ytrain = integerY[train]
      Xtest = X_escalado[test,:]
      Ytest = integerY[test]
      Ytest=Ytest.reshape(len(Ytest))
      Ytrain=Ytrain.reshape(len(Ytrain))
      clf.fit(Xtrain, Ytrain)
      Ytrain_pred = clf.predict(Xtrain)
      Yest = clf.predict(Xtest)
      #Evaluamos las predicciones del modelo con los datos de test
      EficienciaTrain[j] = metrics.accuracy_score(Ytrain, Ytrain_pred)
      EficienciaVal[j] = metrics.accuracy_score(Ytest, Yest)
      f1[j]=metrics.f1_score(Ytest, Yest,average='weighted')   
  print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
  print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
  print("f1-score ",str(np.mean(f1))," +- ",np.std(f1))
  tablaVentanaParzen = completarTablaVentanaParzen(i,tablaVentanaParzen,np.mean(EficienciaVal),np.std(EficienciaVal),np.mean(f1),np.std(f1))

Eficiencia durante el entrenamiento = 0.25+-0.4330127018922193
Eficiencia durante la validación = 0.13766788766788768+-0.23844777601146633
f1-score  0.1375372968401485  +-  0.23822158606281962
Eficiencia durante el entrenamiento = 0.2498982498982499+-0.4328364655463129
Eficiencia durante la validación = 0.15567765567765568+-0.26964160923691316
f1-score  0.15139190182873427  +-  0.2622184658218474
Eficiencia durante el entrenamiento = 0.2271062271062271+-0.39335952406326147
Eficiencia durante la validación = 0.16056166056166057+-0.27810095384042416
f1-score  0.14650600754819118  +-  0.25375584868753653
Eficiencia durante el entrenamiento = 0.17928367928367928+-0.31052844148721626
Eficiencia durante la validación = 0.15964590964590963+-0.2765148267272658
f1-score  0.13298671045560626  +-  0.23033973924056125
Eficiencia durante el entrenamiento = 0.15282865282865282+-0.2647069915515317
Eficiencia durante la validación = 0.15262515262515264+-0.2643545188597188
f1-score  0.11571277681967598

In [55]:
tablaVentanaParzen

,bandwidth,Eficiencia en validacion,Intervalo de confianza de eficiencia,f1-score,Intervalo de confianza de f1-score
0,0.2,0.13766788766788768,0.23844777601146633,0.1375372968401485,0.23822158606281962
1,0.5,0.15567765567765568,0.26964160923691316,0.15139190182873427,0.2622184658218474
2,0.7,0.16056166056166057,0.27810095384042416,0.14650600754819118,0.25375584868753653
3,1.0,0.15964590964590963,0.2765148267272658,0.13298671045560626,0.23033973924056125
4,2.0,0.15262515262515264,0.2643545188597188,0.11571277681967598,0.20042040853655704
5,5.0,0.15262515262515264,0.2643545188597188,0.11571277681967598,0.20042040853655704
6,10.0,0.15262515262515264,0.2643545188597188,0.11571277681967598,0.20042040853655704


# 4. Análisis discriminante cuadrático

In [56]:
tablaAnalisisDiscriminanteCuadratico = pd.DataFrame({
    'regParam' : pd.Series([0.0,0.3,0.6,0.9])})
tablaAnalisisDiscriminanteCuadratico["Eficiencia en validacion"] = ""
tablaAnalisisDiscriminanteCuadratico["Intervalo de confianza"] = ""
regParam=[0.0,0.3,0.6,0.9]
for i in range(4):
  clf = QuadraticDiscriminantAnalysis(reg_param=regParam[i])
  Folds = 4
  EficienciaTrain = np.zeros(Folds)
  EficienciaVal = np.zeros(Folds)
  skf = StratifiedKFold(n_splits=Folds)
  f1 = np.zeros(Folds)
  j = 0
  for train, test in skf.split(X_escalado, integerY):
      Xtrain = X_escalado[train,:]
      Ytrain = integerY[train]
      Xtest = X_escalado[test,:]
      Ytest = integerY[test]
      Ytest=Ytest.reshape(len(Ytest))
      Ytrain=Ytrain.reshape(len(Ytrain))
      clf.fit(Xtrain, Ytrain)
      Ytrain_pred = clf.predict(Xtrain)
      Yest = clf.predict(Xtest)
      EficienciaTrain[j] = metrics.accuracy_score(Ytrain, Ytrain_pred)
      EficienciaVal[j] = metrics.accuracy_score(Ytest, Yest)
      f1[j]=metrics.f1_score(Ytest, Yest,average='weighted')   
  print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
  print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
  print("f1-score ",str(np.mean(f1))," +- ",np.std(f1))
  tablaAnalisisDiscriminanteCuadratico = completarTablaAnalisisDiscriminanteCuadratico(i,tablaAnalisisDiscriminanteCuadratico,np.mean(EficienciaVal),np.std(EficienciaVal),np.mean(f1),np.std(f1))


Eficiencia durante el entrenamiento = 0.17073667073667073+-0.295724588431072
Eficiencia durante la validación = 0.1681929181929182+-0.2913186797834101
f1-score  0.15945546406053548  +-  0.27618496529732056
Eficiencia durante el entrenamiento = 0.17104192104192104+-0.2962532974687914
Eficiencia durante la validación = 0.16697191697191696+-0.2892038436325323
f1-score  0.15449234495960465  +-  0.26758859085049286
Eficiencia durante el entrenamiento = 0.16778591778591778+-0.29061373439978416
Eficiencia durante la validación = 0.16483516483516483+-0.28550288036849625
f1-score  0.14636980302711047  +-  0.25351993553680424
Eficiencia durante el entrenamiento = 0.15873015873015872+-0.2749286996141075
Eficiencia durante la validación = 0.15567765567765568+-0.26964160923691316
f1-score  0.12446570422516796  +-  0.21558092351783123


In [57]:
tablaAnalisisDiscriminanteCuadratico

,regParam,Eficiencia en validacion,Intervalo de confianza,Intervalo de confianza de eficiencia,f1-score,Intervalo de confianza de f1-score
0,0.0,0.1681929181929182,,0.2913186797834101,0.15945546406053548,0.27618496529732056
1,0.3,0.16697191697191696,,0.2892038436325323,0.15449234495960465,0.26758859085049286
2,0.6,0.16483516483516483,,0.28550288036849625,0.14636980302711047,0.25351993553680424
3,0.9,0.15567765567765568,,0.26964160923691316,0.12446570422516796,0.21558092351783123


# 5. Gradient boosting tree

In [70]:
tablaBoostingTree = pd.DataFrame()
tablaBoostingTree["n_estimators"] = ""
tablaBoostingTree["learning_rate"] = ""
tablaBoostingTree["max_depth"] = ""
tablaBoostingTree["Eficiencia en validacion"] = ""
tablaBoostingTree["Intervalo de confianza"] = ""


n_estimators=[300,500,1000,]
learning_rate=[0.3,0.5,0.7]
max_depth=[1,3,5]
pos=0
for i in range(3):
  for z in range(3):
    for k in range(3):
      clf = GradientBoostingClassifier(n_estimators=n_estimators[z], learning_rate=learning_rate[k],max_depth=max_depth[i], random_state=0)
      Folds = 4
      EficienciaTrain = np.zeros(Folds)
      EficienciaVal = np.zeros(Folds)
      skf = StratifiedKFold(n_splits=Folds)
      f1 = np.zeros(Folds)
      j = 0
      for train, test in skf.split(X_escalado, integerY):
          Xtrain = X_escalado[train,:]
          Ytrain = integerY[train]
          Xtest = X_escalado[test,:]
          Ytest = integerY[test]
          Ytest=Ytest.reshape(len(Ytest))
          Ytrain=Ytrain.reshape(len(Ytrain))
          clf.fit(Xtrain, Ytrain)
          Ytrain_pred = clf.predict(Xtrain)
          Yest = clf.predict(Xtest)
          #Evaluamos las predicciones del modelo con los datos de test
          EficienciaTrain[j] = metrics.accuracy_score(Ytrain, Ytrain_pred)
          EficienciaVal[j] = metrics.accuracy_score(Ytest, Yest)
          f1[j]=metrics.f1_score(Ytest, Yest,average='weighted') 
          j += 1  
      print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
      print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
      print("f1-score ",str(np.mean(f1))," +- ",np.std(f1))
      tablaBoostingTree=completarTablaBoostingTree(pos,tablaBoostingTree,np.mean(EficienciaVal),np.std(EficienciaVal),n_estimators[z],learning_rate[k],max_depth[i],np.mean(f1),np.std(f1))
      pos+=1

Eficiencia durante el entrenamiento = 0.781949531949532+-0.007237143631291538
Eficiencia durante la validación = 0.7448107448107448+-0.038213632123391496
f1-score  0.7283591874359728  +-  0.03676154284602545
Eficiencia durante el entrenamiento = 0.7969067969067969+-0.011601296551680277
Eficiencia durante la validación = 0.7387057387057387+-0.03797883031745002
f1-score  0.7259814617676325  +-  0.03718059952287511
Eficiencia durante el entrenamiento = 0.8137973137973138+-0.006489710508385034
Eficiencia durante la validación = 0.7313797313797314+-0.03439454461783461
f1-score  0.7215598488202637  +-  0.033004275169576575
Eficiencia durante el entrenamiento = 0.7921245421245421+-0.00962009411491537
Eficiencia durante la validación = 0.7417582417582418+-0.03873192167414382
f1-score  0.7275948041443806  +-  0.03881565756410568
Eficiencia durante el entrenamiento = 0.8161375661375662+-0.0058015406146309885
Eficiencia durante la validación = 0.7356532356532357+-0.03143337157962394
f1-score  0.7

In [71]:
tablaBoostingTree

,n_estimators,learning_rate,max_depth,Eficiencia en validacion,Intervalo de confianza,f1-score,Intervalo de confianza de f1-score
0,300,0.3,1,0.7448107448107448,0.038213632123391496,0.7283591874359728,0.03676154284602545
1,300,0.5,1,0.7387057387057387,0.03797883031745002,0.7259814617676325,0.03718059952287511
2,300,0.7,1,0.7313797313797314,0.03439454461783461,0.7215598488202637,0.033004275169576575
3,500,0.3,1,0.7417582417582418,0.03873192167414382,0.7275948041443806,0.03881565756410568
4,500,0.5,1,0.7356532356532357,0.03143337157962394,0.7252039478361147,0.031068274509417487
5,500,0.7,1,0.7255799755799757,0.030037312499071576,0.7174415745985367,0.029365577780117228
6,1000,0.3,1,0.7310744810744811,0.031277358987499136,0.7209011803572587,0.031120998240496956
7,1000,0.5,1,0.7271062271062272,0.026715941986619116,0.7191758273012587,0.026538672010077726
8,1000,0.7,1,0.721916971916972,0.020565354411112912,0.714961119732852,0.021036038934216742
9,300,0.3,3,0.7338217338217339,0.034351171908755064,0.729392315169478,0.0352887929822425
